In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression


from IPython.display import display


In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors ='coerce')        #coerce -> replace all nonnumeric values with NaN
df.TotalCharges = df.TotalCharges.fillna(0)
df.columns = df.columns.str.lower().str.replace(' ', '_')
string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [ ]:
df.churn = (df.churn == 'yes').astype(int) 

In [ ]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

# now spliting it in train and validation
df_train, df_val = train_test_split(df_train_full, test_size= 0.33, random_state= 11)

y_train = df_train.churn.values
y_val = df_val.churn.values

del df_train['churn']
del df_val['churn']

In [ ]:
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.churn)

In [ ]:
df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')
df_mi

In [ ]:
df_train_full[numerical].corrwith(df_train_full.churn)

In [ ]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

In [ ]:
X_train = dv.transform(train_dict)

In [ ]:
model = LogisticRegression(solver='liblinear',random_state=1)
model.fit(X_train, y_train)

In [ ]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)
churn = y_pred>=0.5
churn


In [ ]:
(y_val == churn).mean()

In [ ]:
customer = {
    'customerid':'83844-dfsdf',
    'gender': 'female',
    'seniorcitizen':0,
    'partner':'no',
    'dependents':'no',
    'tenure': 41,
    'phoneservice':'yes',
    'multiplelines':'no',
    'internetservice':'dsl',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'yes',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'one_year',
    'paperlessbilling':'yes',
    'paymentmethod':'bank_transfer_(automatic)',
    'monthlycharges':  79.85,
    'totalcharges': 3320.75    
}

In [ ]:
X_test = dv.transform([customer])
#model.predict_proba(X_test)
model.predict_proba(X_test)[0,1]


In [ ]:
customer_2 = {
    'customerid':'83844-dfsdf',
    'gender': 'female',
    'seniorcitizen':0,
    'partner':'yes',
    'dependents':'yes',
    'tenure': 10,
    'phoneservice':'no',
    'multiplelines':'no',
    'internetservice':'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'no',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling':'yes',
    'paymentmethod':'electronic_check',
    'monthlycharges':  400,
    'totalcharges': 500    
}

In [ ]:
X_test_2 = dv.transform([customer_2])
model.predict_proba(X_test_2)